In [ ]:
!pip install sklearn_crfsuite

In [ ]:
import pandas as pd

from sklearn_crfsuite import CRF
from sklearn_crfsuite import metrics

In [ ]:
def extract_sentence_features(df):
    sentence_length = len(df)

    for i in range(sentence_length):
        token = df.iloc[i]['Token']

        features = {
            'bias': 1.0,
            'token.lower()': token.lower(),
            'token.isupper()': token.isupper(),
            'token.istitle()': token.istitle(),
            'token.isdigit()': token.isdigit()
        }

        if i > 0:
            previous_token = df.iloc[i-1]['Token']
            features.update({
                'previous_token.lower()': previous_token.lower(),
                'previous_token.isupper()': previous_token.isupper(),
                'previous_token.istitle()': previous_token.istitle(),
                'previous_token.isdigit()': previous_token.isdigit()
            })
        else:
            features['BOS'] = True

        if i < sentence_length - 1:
            posterior_token = df.iloc[i+1]['Token']
            features.update({
                'posterior_token.lower()': posterior_token.lower(),
                'posterior_token.isupper()': posterior_token.isupper(),
                'posterior_token.istitle()': posterior_token.istitle(),
                'posterior_token.isdigit()': posterior_token.isdigit()
            })
        else:
            features['EOS'] = True

        yield features

In [ ]:
def prepare(df, include_y=False):
    X, y = [], []
    for _, group_df in df.groupby(['PMID_Type', 'Sentence_Index']):
        X.append(list(extract_sentence_features(group_df)))
        if include_y:
            y.append(group_df['Tag'])
    if include_y:
        return X, y
    return X

In [ ]:
train_df = pd.read_csv('/kaggle/input/chemdner-iob-annotated-chemical-named-etities/training.csv')
validation_df = pd.read_csv('/kaggle/input/chemdner-iob-annotated-chemical-named-etities/validation.csv')
evaluation_df = pd.read_csv('/kaggle/input/chemdner-iob-annotated-chemical-named-etities/evaluation.csv')

In [ ]:
train_df.head()

In [ ]:
train_df.info()

In [ ]:
validation_df.info()

In [ ]:
evaluation_df.info()

In [ ]:
train_df['Token'].fillna('', inplace=True)
validation_df['Token'].fillna('', inplace=True)
evaluation_df['Token'].fillna('', inplace=True)

In [ ]:
X_train, y_train = prepare(train_df, include_y=True)
X_validation, y_validation = prepare(validation_df, include_y=True)
X_evaluation, y_evaluation = prepare(evaluation_df, include_y=True)

In [ ]:
len(X_train), len(y_train)

In [ ]:
crf = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

In [ ]:
crf.fit(X_train, y_train)

In [ ]:
y_pred = crf.predict(X_validation)

In [ ]:
print(metrics.flat_classification_report(y_validation, y_pred))